# Convolutional Neural Network

### Importing the libraries

In [6]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install keras

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
tf.__version__

'2.16.1'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [12]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('datasets/facemask_detection/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 6362 images belonging to 2 classes.


### Preprocessing the Test set

In [14]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('datasets/facemask_detection/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 1191 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [17]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [19]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

C:\Users\karthikgokul\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [21]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [23]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [25]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [27]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [29]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [32]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [34]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 2)

Epoch 1/2


C:\Users\karthikgokul\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  7/199 ━━━━━━━━━━━━━━━━━━━━ 4:39 1s/step - accuracy: 0.4469 - loss: 1.0432

C:\Users\karthikgokul\anaconda3\Lib\site-packages\PIL\Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


199/199 ━━━━━━━━━━━━━━━━━━━━ 305s 1s/step - accuracy: 0.6868 - loss: 0.5725 - val_accuracy: 0.7783 - val_loss: 0.5070
Epoch 2/2
199/199 ━━━━━━━━━━━━━━━━━━━━ 90s 438ms/step - accuracy: 0.8793 - loss: 0.3066 - val_accuracy: 0.9118 - val_loss: 0.2505


## Part 4 - Making a single prediction

In [36]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('datasets/facemask_detection/single_prediction/check_4.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'Without Mask'
else:
  prediction = 'With Mask'

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 632ms/step


In [37]:
print(prediction)

Without Mask
